##### [Cryptographic Hash Functions](#ca)

- [What we would expect from a cryptograpic alrorithm (CA)](#expect)


##### [Vulunerablility and types of attacks ](#vul)

- [Which type of attack would have a more catastrophic impact](#catast)
- [Broken CAs](#brok)

##### [Different types of algorithms and their use cases ](#types_alg)

##### [Discussion](#discussion)

##### [Our implementation of SHA-256](#our_imp)


##### [Building the SHA-256 cryptographic algorithm](#b_algo)

- [Python bitwise operators](#bwise)
- [Operators to be used for the SHA-256  development](#opers)
    - [Exlusive or XOR (^)](#xor)
    - [Bitwise right shift (>>)](#rsh)
    - [Bitwise left shift (<<)](#lsh)
    - [Bitwise OR operation ( | )](#or)
    - [Bitwise NOT operation (~)](#not)

- [Constant values to be used for the SHA-256 ](#const)
    - [The $K$ constants](#kconst)
    - [The $H$ constants](#hconst)

- [The sha256 methods to be used for the algorithm](#meth)
    - [Mod $2^{32}$ Addition](#mod32)
    - [Righ rotation ($r\_rot(x, steps)$)](#rrot)
    - [Preproccessing the input message ($padding()$)](#padding)

- [Logical functions used for the cryptographic algorithm's main method](#logic)
    - [Choice function ($choice(x, y, z)$)](#choise)  
    - [Majority function ($majority(x, y, z)$)](#maj)
    - [$\Sigma_0$ function](#S0)
    - [$\Sigma_1$ function](#S1)
    - [$\sigma_0$ function](#s0)
    - [$\sigma_1$ function](#s1)
    - [The main hashing function](#comp)

- [Combining everything to build the SHA-256  algorithm](#comb)
    - [Comparison to the python hashlib](#compare)


##### [References](#ref)




# Cryptographic Hash Functions
<a id="ca"></a>

Hashing is an important component of the modern digital world. Its extensive use has multiple applications but most importantly allows secure network communications, builds trust between users, organisations, governments and enables a smooth delivery of many interactions of the society. 

Some of the hashing applications in day-to-day life and technology are the following:


- Password verification and message digestion: Web sites verify the user's identity by storing and comparing the hashed version of a password without the need of storing the actual password which would come with high risks. Another example the cryptographic algorithm can digest a chunk of data (e.g. a contract or a ledger) and produce signatures. A common application is the cryptographic component of the cryptocurrency.

- Programming languages' data structures: Hashing data structures are used by computers to perfrorm operations like  comparison or identification of duplicates.

## What we would expect from a cryptographic algorithm (CA)
<a id="expect"></a>

An 'ideal' CA should have some essential properties. By principal, CAs have to be one-way. That means that one can easily produce an output of specific length $L$ (hash value) from an input but it should be very difficult to reverse the function and produce the initial message from the output. Another property that a CA should have is that for a minor alteration on the input message, the hashed value should change drastically<sup>[[1]](#ref)</sup> (Avalanche effect). Finally, for an 'ideal' CA it should be difficult to identify two different inputs that result into the same hash value. Although it's possible that two messages have the same hash value<sup>[[2]](#ref)</sup> (Pigeonhole principle), this is not something likely to occur 'naturally' (without effort). In terms of message digesting, a CA has to have a 'balanced' speed. This means that it has to be fast enough for a user to digest big documents but slow enough to discourage brute-force attacks. For example, a CA to be used for hashing passwords should be slower than one used for larger message digestion.

As the term 'difficult' mentioned above is quite abstract, the effort required to compromise the two essential characteristics of an ideal CA that can be expressed mathematically. It should take $2^{L/2}$ attempts to find two messages($M$ and $M'$) that generate the same hash value (collision-free property) and for a known $M$ should take $2^L$ attempts to find a $M'$ that has the same hash value (preimage property).

Under the assumption that such a CA wouldn't have any weaknesses, it would only be possible to undermine its integrity by brute-force. This means that the attacker should try all the possible combinations to either compromise the 'one-way' or 'collision-free' property of the function. A CA will be considered functional as long as an attack requires a disproportional amount of time and resources (computational power).

# Vulnerablility and types of attacks 
<a id="vul"></a>

A CA would be considered vulnerable when it is worth the attacker's time and resources to compromise one of its properties. Therefore:

1. An attack to the collision-free property would be practical when the attacker can identify a $M$ and $M'$ with the same hash with less that $2^{L/2}$ attempts.


2. An attack to the one-way property would be possible when the attacker can either:
    - Find a $M$ for a known hash value in less that $2^L$ attempts (first preimage attack)
    - For a known $M$ find a $M'$ with the same hash value in less than $2^L$ attempts (second preimage attack)
    
 
## Which type of attack would have a more catastrophic impact
<a id="catast"></a>

The lifecycle of a CA comes into and end when its properties are no longer valid. Although, the 'collision-free' characteristic of a CA is important, the preimage attacks could be devastating for the reputation and the security an algorithm can provide. By definition, in a collision attack scenario the attacker is free to choose between two random messages that share a common hash value ($2^{L/2}$ attempts required). In the second preimage attack scenario, the task is to find a message that its key matches the one of an already known message (a more difficult task with complexity $2^L$). If the attacker finds a 'practical' way to achieve this then the function ceases to be one-way. 

The two attacks might have similarities but are different concept-wise. 

For example, when an attacker has the freedom to forge two contracts with the same signature, they can use the two documents interchangeably on his favour to perform a fraud. This would be a collision attack.

In a different case, when an attacker wills to forge a signature for a given message, the second preimage is of great importance. The task is much harder as it requires finding an identical signature for a given data message.


From the nature of the examples above we could come up with two conclusions: 


 
- A second preimage is essentially a version of collision that is harder to solve. If the second preimage resistance of a CA is compromised, then the collision resistance is compromised as well (but not the other way around). 


- If the second preimage is attacked, the impact would be far more devastating as a great range of protocols and certificates will be affected. In this case, the one-way property of a CA is completely eliminated and the function is considered completely broken. 

## Broken CAs (MD2)
<a id="brok"></a>

Looking in to the IETF's<sup>[[3]](#ref)</sup> 'Request for Comments'<sup>[[4]](#ref)</sup> (a memorandum on Internet standards), one can gain some good understanding about the introduction and the end of life of the MD2 algorithm. 

In [August 1989](https://datatracker.ietf.org/doc/html/rfc1115) MD2 was introduced by Ron Rivest (RSA<sup>[[7]](#ref)</sup>) for use on  privacy-enhanced electronic mail.  


In [April 2002](https://datatracker.ietf.org/doc/html/rfc3279) were expressed some concerns about the integrity of the algorithm as the checksum operation was the only obstacle for a succesful attack. Thus it was recommended that MD2 could be still used to verify existing signatures, as the ability to find collisions in MD2 would not enable an  attacker to find new messages having a previously computed hash value. It was also recommended that the algorithm shouldn't be used in new applications.


In a [March 2011](https://datatracker.ietf.org/doc/html/rfc6149#ref-KMM2010) RFC a preimage attack of $2^{73}$ complexity was confirmed<sup>[[5]](#ref)</sup>. The MD2 was not one-way anymore and shouldn't be used for digital signatures. 


One of the basic concepts that is making attacks feasible is the birthday paradox<sup>[[6]](#ref)</sup>. Based on the statistical fact that the attempts needed to compromise a CA property are less than just using brute force, the attackers were able to apply several scenarios to make MD2 invertible. 


Generally, an attack might not be computationally practical but still can reveal weaknesses of cryptographic algorithm. As noted by Bruce Schneier<sup>[[12]](#ref)</sup>, if an algorithm is proved to be attacked in fewer attempts than brute force, it is still considered as broken even if this attack is not possible in practice.


# Different types of algorithms and their use cases (PBKDF2 and sha256)
<a id="types_alg"></a>


There are several hashing algorithms developed over the years with each one having the same properties but different design and characteristics. In this section we will see how PBKDF2 and SHA-256 are different from each other and why they perform better in different tasks.

PBKDF2 comes from RSA's 'Public-Key Cryptography Standards'<sup>[[8]](#ref)</sup> (PKCS) series. As it was stated in [RFC 2898](https://www.ietf.org/rfc/rfc2898.txt), since passwords were often chosen from a relatively small space *(users tend to select similar common words as passwords. Additionally, passwords are usually relatively short)*, special care was required from the cryptographic operations to defend against search attacks. The main focus of PBKDF2 is to generate a hash version of passwords since the cryptosystems cannot store directly passwords.       

The two main characteristics of PBKDF2 are:
     
   _**a)** Incorporates a 'salt value'<sup>[[9]](#ref)</sup>(password hashes use a salt so that identical passwords won't map to the same hash)_ 
   
   _**b)** It is by design slower in comparison to other algorithms. For example, for a sample run of 40 hashes, using a Linux Mint 64 bit operating system on a Core i5 3340M at 2.7GHz, the SHA-256  performs 126,323 hashes per second when PBKDF2 performs only 5,026<sup>[[10]](#ref)</sup>_
   
   _**c)** Incorporates an adjustable  [work factor](https://csrc.nist.gov/glossary/term/work_factor)._


The SHA-256  belongs to the SHA-2 family of algorithms with digest of 256 bits. SHA-2 was first published by the [National Institute of Standards and Technology](https://www.nist.gov) (NIST) as a Federal Information Processing. SHA-256 is commonly used when the input message is relatively large. For example, Linux distribution operating systems (like Debian) are using SHA-256 to authenticate packages<sup>[[11]](#ref)</sup>. Another common application is on cryptocurrencies. The main characteristics of SHA-256. SHA-256 compared to PBKDF2:

   _**a)** Does not have a salt value (although a similar effect can be achieved by prepending the salt to the input message)_ 
   
   _**a)** It's fast by design_
   
Generally, from the use cases for each algorithm we can see that in the PBKDF2 case the resulting output isn't used as key but as hash value and it's also called a password hash. It is different from SHA-256 as it contains a salt value and a work factor. The salt value and the slowness of PBKDF2 makes it more resistant to brute force attacks, using a [dictionary](https://en.wikipedia.org/wiki/Dictionary_attack) of common passwords, or using a [rainbow table](https://en.wikipedia.org/wiki/Rainbow_table) (a pre-compiled list of hash values). On the other hand side, the SHA-256 is used by Bitcoin for verifying transactions and calculating proof of work. Because of its speed, SHA-256 can provide a good balance of collision-avoidance and performance.

# Discussion
<a id="discussion"></a>


Apart from all the methodological aspects of each type of attacks, it is also useful to take into account their requirements in terms of resources<sup>[[13]](#ref)</sup>. Three major parameters that play an important role in the efficiency of an algorithm are:

- The computational steps for the attack

- The required memory storage

- The length of the input message and the hash value

These parameters along with the particularity of each algorithm (i.e., speed) make the evaluation of characterising  an attack feasible a difficult task. In the field of cryptanalysis, the order of the magnitude for an attack might be summarised in a single number (e.g. $2^L$) but how practical such an attack is would depend on other factors as well. One of the main concerns is how much the increase in computer power (according to Gordon Moore<sup>[[18]](#ref)</sup> "The number of transistors incorporated in a chip will approximately double every 24 months.") enables criminal activity against cryptography. Apart from the increased access to supercomputing that is now possible through the cloud services, it is also investigated how quantum computers will affect security in the future (Fred Pipe<sup>[[14]](#ref)</sup>).


Finally, since cryptography lies in the heart of cryptocurrency as a mean for distributing trust through the concept of proof of work, there is also an environmental issue raised. Proof of work is very demanding in terms of renewable energy and consequently has an impact to climate change via the carbon emissions produced. To tackle this problem, some cryptocurrencies introduced the concept of proof of stake as an alternative to proof of work. Although proof of stake has significantly less requirements in terms of energy, there is a debate on whether the distributed trust concept is compromised under the assumption that more wealthy miners will be able to earn more.

# Our implementation of SHA-256
<a id="our_imp"></a>

In the second part of this work we will attempt to develop the SHA-256 algorithm. We will dive into the details of how all of its components work to finally combine all the [steps required](#comb) (link to a following paragraph) to replicate the algorithm.

# Building the SHA-256  cryptographic algorithm
<a id="b_algo"></a>

In this section we will be developing the SHA-256  cryptographic algorithm as described in the Information Warefare Site<sup>[[15]](#ref)</sup>. To achieve that we will construct and explain all the operations and components of the algorithm and we will finally implement it as a python class.

## Python bitwise operators 
<a id="bwise"></a>

As we know already, the SHA-256  cryptographic algorithm digests our original text 'translated' into bits. Therefore we will be working with python bitwise operators. The bitwise operations take one or two arguments and instead of operating on them as a whole, they process them bit by bit.

For example, when arithmetic operators are applied on numerical values, the bitwise orepators are used to compare the binaries beneath these values.

In [1]:
# Arithmetic 'add'(+) operator

5 + 7

12

In [2]:
# Bitsiwe 'and'(&) operator  

5 & 7

5

This might not be immediately obvious but it makes more sense if we see the binaries of 5 and 7.

In [3]:
print ('The binary representations of 5 and 7 are respectively {} and {}'.format(bin(5)[2:], bin(7)[2:]))

The binary representations of 5 and 7 are respectively 101 and 111


So, we can write the bitwise operation (&) using the binaries:

In [4]:
bin(0b101 & 0b111)

'0b101'

To better understand this output we can see the python's documentation<sup>[[16]](#ref)</sup> for the 'and' (&) operator. The bits of 5 and 7 are compared one by one by the & operator to give us the following results:

1&1  -> 1

0&1  -> 0

1&1  -> 1


The binary that we obtain after the oparation is 101, which corresponds to 5:

In [5]:
0b101 & 0b111

5

Therefore, arithmetically the bitwise 'and' (&) can be expressed by:

$(a  \&  b)_i = a_i \cdot b_i$

## Operators to be used for the SHA-256  development
<a id="opers"></a>

Along with the 'and' operator we will use the following operators to develop our algorithm:

### Exlusive or XOR (^)
<a id="xor"></a>

This operator returns a 1 for an odd result of the sum of the corresponding bits and a 0 if the sum is even (or 0).

Example:

In [6]:
bin(0b101 ^ 0b111)

'0b10'

1^1  -> 0

0^1  -> 1

1^1  -> 0

This can be mathematically expressed by the sum of the coresponding bits modulo 2:

$(a $^$  b)_i = (a_i + b_i)mod2$

### Bitwise right shift (>>)
<a id="rsh"></a>

This operator shifts the bits of a word $n$ steps rightwise. For example:

1 0 0 1 0 0 1 1 1 
     
     >> 2
      =
0 0 1 0 0 1 0 0 1 

which essentially equals  to 1 0 0 1 0 0 1 as the zeros in the begining of a word could be excluded without alternating its meaning.

It is important to note that by performing $n$ bits right shift we lose $n$ bits of 'information' from the right end of our word.

Python example:

In [7]:
bin(0b100100111 >> 2)

'0b1001001'

This can be mathematically expressed as:

$a >> n = [\frac{a}{2^n}]$

### Bitwise left shift (<<)
<a id="lsh"></a>

The left shift operator shifts the input word $n$ bits to the left. In order to maintain the words length, it adds $n$ number of zeros to the right.

Example:

In [8]:
bin(0b100100111 << 2)

'0b10010011100'

A mathematical expression would be:

$a << n = a \cdot 2^n$

### Bitwise OR operation ( | )
<a id="lsh"></a>

The bitwise 'or' operator returns a 1 if at least one of the two bits compared is 1. For example:

In [9]:
bin(0b101 | 0b111)

'0b111'

1|1  -> 1

0|1  -> 1

1|1  -> 1

This can be mathematically expressed by the union of the two operators:

$(a | b)_i = a_i + b_i - (a_i \cdot b_i)$

### Bitwise NOT operation (~)
<a id="not"></a>

This operation takes only one argument and reverses its bits (1s are turned to 0s and 0s to 1s) 

~1  -> 0

~0  -> 1

~1  -> 0

This could be expressed as:

$ \sim a_i = 1 - a_i$ (bitwise complement)

## Constant values to be used for the SHA-256 
<a id="const"></a>

The following two sets of constant values $H$ and $K$ will be used to mix the message that we will later pass into the main compression function of our algorithm. For this imlementation, the constants from the SHA-256  description paper are used. The following paragraphs explain how these constants are produced.

### The $K$ constants
<a id="kconst"></a>

$K_{1-64}$ is a sequence of constant words given in hexadecimal which are produced by the first 32 digits of the fractional part of the cube root of the first 64 prime numbers.

For example, $K_1$ will derive from the hexadecimal of the first 32 decimals of $\sqrt[3]{2}$. Similarily, $K_2$ will derive from $\sqrt[3]{3}$ and so on up to $\sqrt[3]{311}$ .

$K_{1-64}$ = [<mark>0x428a2f98</mark>, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
              0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
              0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
              0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
      0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
      0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
      0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
      0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
      0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
      0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
      0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
      0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
      0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
      0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
      0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
      0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

As we can see, we can reproduce the float from which <mark>'428a2f98'</mark> was derived.

In [10]:
# From hexadecimal to float
decimals = []
for (i, x) in enumerate('428a2f98'):
    decimals.append(int(x,16)/(16**(i+1)))
print (sum(decimals))

0.2599210496991873


In [11]:
# The cube root of 2
# Only the digits after the fraction were used for the hexadecimal above
# therefore we deduct 1 (-1)


print (2**(1/3)-1) # As this is an example the result is almost similar. 
                   # Some long decimal calculations led to 
                   # slight alternation of the final value
                   # For the implementation, the values from the SHA-256 description paper were used

0.2599210498948732


### The $H$ constants
<a id="hconst"></a>

For the initial 8 hash words to be produced $H_{1-8}$ is used the hexadecimal of the fractional part of the square roots of the first 8 prime numbers $\sqrt{2}$, $\sqrt{3}$ .. and so on.

$H_{1-8}$ = [<mark>0x6a09e667</mark>, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
      0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]

An example of how <mark>'6a09e667'</mark> was derived from the fractional part of $\sqrt{2}$ :

In [12]:
# From hexadecimal to float
decimals = []
for (i, x) in enumerate('6a09e667'):
    decimals.append(int(x,16)/(16**(i+1)))
print (sum(decimals))

0.41421356215141714


In [13]:
print (2**(1/2)-1) # again this is almost equal due to operations with many decimals

0.41421356237309515


## The SHA-256 methods to be used for the algorithm
<a id="meth"></a>

Based on the python's bitwise operations we will develop the following methods needed for the the cryptographic alogorithm:

### Mod $2^{32}$ Addition
<a id="mod32"></a>

This operation for a word $x$ can be written as:

$x$ $mod$ $n = c$  

where:


$0 \leqslant c < n$  

and 

$(x  - c)$ is a multiple of $n$

In bitwise python to perform the $mod$ $n$ for $n=32$ we add   *0xFFFFFFFF* (32 bits in hexadecimal). For example, the Mod $2^{32}$ addition of the word:

In [14]:
x = 0b1001100001000011000000000010110010011000010000110000000000101

will be:

In [15]:
print (bin(x & 0xFFFFFFFF))

0b10010011000010000110000000000101


### Righ rotation ($r\_rot(x, steps)$)
<a id="rrot"></a>

This method takes as an input a bit 'word' and rotates the bits righwise. Similar to the right shift, this function moves the digits to the right but instead of replacing the last digit of the word with a 0 in the start (left), it actually moves it at the start of the sequence.

For example:

for the **word** = 1 0 0 1 0 1 1 1

if we rotate to the right by **steps** = 2 

then 

**r_rot**(word, steps) ->  1 1 1 0 0 1 0 1

To build a python function for right rotation we can perform the following bitwise operation steps.

To rotate, for example the 32-bit word:

In [16]:
x = 0b1001100001000011000000000010110

2 steps to the right

In [17]:
steps = 2

We initially shift our word two steps rightwise

In [18]:
print (bin(x >> steps))

0b10011000010000110000000000101


Since we need to maintain the infrormation we have lost with the right shift we perfrorm a 32 - 2 left shift to our initial word.

In [19]:
print (bin(x << (32 - steps)))

0b1001100001000011000000000010110000000000000000000000000000000


We take the union of right and left shift:

In [20]:
print (bin(((x >> 2) | (x << (32 - 2)))))

0b1001100001000011000000000010110010011000010000110000000000101


Finally, we perfrom Mod $2^{32}$ addition to keep a $n = 32$ multiple of the word which corresponds to the rightwise rotated initial word.

In [21]:
f32 = 0xFFFFFFFF

print (bin(((x >>steps) | (x << (32 - steps))) & f32))

0b10010011000010000110000000000101


We can now write everything as a function so we can use it in our algorithm development:

In [22]:
f32 = 0xFFFFFFFF

x = 0b1001100001000011000000000010110

def r_rot(x, steps):
    return ((x >> steps) | (x << (32 - steps))) & f32

bin(r_rot(x, 2))

'0b10010011000010000110000000000101'

### Preproccessing the input message ($padding()$)
<a id="padding"></a>

The SHA-256 hashes data in chunks of 512-bits at the time. Therefore we need to make sure that the input's message length is a multiple of 512-bits. If the lentgh is shorter than 512-bits we pad it to 512, otherwise we pad it to the next closest multipe (1024). Our SHA-256 implementation, in the event that the message is longer than 64 bytes, will digest all the message blocks multiples and the remainders will be sent for padding in order to be consumed in a last itteration by the compression function.


Let's take an example where the input message is 'abc'. The length of this message is 3 bytes (< 64). The objective here is to pad our message to 64 bytes. To do so, we use the original message (3 bytes), we add a 1 as a seperator and we fill the middle part of our pad with zeros up to the lenght of 448th bits. The last 64 bits are preserved and filled with the length of the binary representation of the initial message.

(In the event of our message being between 56 and 64 bytes it is padded in a similar manner to 128 bytes as the minimum length of a pad can be 9 bytes as we will see later in the implementation)

We can start the construction of our padding function by using 'abc' as an input message.

In [23]:
input_message = 'abc'

The length of the message will be:

In [24]:
inputs_len = len(input_message)
print (inputs_len)

3


To calculate the length of the message in bits we multiply the number of its characters (integer) by $2^3$ since in ascii every character can be expressed in 8-bit. Generally, the bitwise left shift operation $x << y$ adds $y$ zeros on the right of the binary. This can be expressed mathematically as $x = 2^y$. Therefore to compute the length of our 8-bit ascii message we multiply its length by $2^3$ (similar to $<< 3$).

In [25]:
lenght = inputs_len << 3
print (lenght)

24


In the following step we use the *struct* package to handle and convert our binary data. We pass the parameter ">Q" in the struch function (where *>* denotes our preference to store the values in a big-endian manner as described in the SHA-256 description paper and "Q" denotes the format which in this case is an integer of length 8 (see python's documentation<sup>[[17]](#ref)</sup>).

We produce a 64 bits (8 bytes) binary that ends with the length of the input message and has 0s in the front.

In [26]:
import struct

lenght_pad = struct.pack('>Q', lenght)
print (lenght_pad)

b'\x00\x00\x00\x00\x00\x00\x00\x18'


As we can see, the 7 bytes at the front are 0s and the last one represents the length of the message.

In [27]:
print ([ord(i) for i in '\x00\x00\x00\x00\x00\x00\x00\x18'])

[0, 0, 0, 0, 0, 0, 0, 24]


To start our pad with a 1 we can use the binary representation of the integer 128. This way we can have an 8 bit word that starts with 1 and the 7 following digits are 0.

In [28]:
ord('\x80')

128

In [29]:
bin(ord('\x80'))

'0b10000000'

So far we have the left part of our pad that starts with 1 ('\x80' --> **1 byte**), the last part that ends with the message length ('\x00\x00\x00\x00\x00\x00\x00\x18' --> **8 bytes**) and we also know the length of our input message ('abc' --> **3 bytes**). To generate an appropriate padding we need to fill the middle part of our pad with zeros ('\x00') to achive a length of 64 bytes.

Therefore, we are looking for a number of zeros that satisfies the following condition:

$message$ $lenght$ $+ 1 + number$ $of$ $zeros + 8 = 64$

or 


 $number$ $of$ $zeros = 55 -$ $message$ $lenght$
 
In this case the number of zeros we need to add is 52.

In [30]:
zeros = 55 - inputs_len
print (zeros)

52


Finally, our pad will be:

In [31]:
print (b'\x80' + (b'\x00' * zeros) + lenght_pad)

b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18'


As we can see the input message added to the pad generates a 64 bytes block message, ready to be digested by our hashing function.

In [32]:
# We add 'abc' + pad

len(b''.join([input_message.encode('ascii'), b'\x80' + (b'\x00' * zeros) + lenght_pad]))

64

Based on this example we can now build the function. We need though to add an if statement for the eventuallity that our message is between 55 and 64 bytes. As the minimun lenght of the pad can be 9 bytes (one byte for the 1 on the left and 8 bytes for the lenght of the mesage on the right) if our message is between 55 and 64 bytes we add a number of 0s to pad the it to 128 bytes (the mext 64 multiple). 

In [33]:
import struct

def padding(msg_lenght):

    length_pad = struct.pack('>Q', msg_lenght << 3)

    mod64 = msg_lenght & 0x3F  

    if mod64 < 56:
        zeros = 55 - mod64
    else:
        zeros = 119 - mod64 # If the message is between 55 and 64 we pad to 128 by adding more zeros

    return b'\x80' + (b'\x00' * zeros) + length_pad

In [34]:
# 'abc' example
padding(inputs_len)

b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18'

## Logical functions used for the cryptographic algorithm's main function
<a id="logic"></a>

There are six logical functions used by SHA-256 for compressing the input message after preproccessing it. As we have already defined all our operations in python we can 'transcript' the following functions and explain what each of them does.

![Logic](img/logical_functions.png)

### Choice function ($choice(x, y, z)$)
<a id="choise"></a>

This function takes 3  words as an input and uses the one to make a decision on how to choose a bit from the other two to include it in the output. In the following example we use $z$ to make a decision. When $z$ is 1 then the $y$ corresponding bit value is selected and when $z$ is 0 the $x$ bit value is selected.

In [35]:
# Example of inputs
z = 0b1110101
y = 0b1001111
x = 0b1010110

def choice(x, y, z):
    return (x & y) ^ (~x & z)

print (bin(choice(x, y, z)))

0b1100111


### Majority function ($majority(x, y, z)$)
<a id="maj"></a>

This function takes 3 words as arguments, and returns the dominant bit value for each position (majority). Here is a python implementation:

In [36]:
# Example of inputs
x = 0b1010110
y = 0b1001111
z = 0b1110101

def majority(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

print (bin(majority(x, y, z)))

0b1010111


### $\Sigma_0$ function
<a id="S0"></a>

The $\Sigma_0$ function , as defined in the SHA-256  description paper, takes as an input a word, then generates three different right rotated versions of its bits (by 2, 13 and 22 steps). It then performs an exclusive or to the three rotated words to generate an output. 

In [37]:
def S0(x):
    return r_rot(x, 2) ^ r_rot(x, 13) ^ r_rot(x, 22)

### $\Sigma_1$ function
<a id="S1"></a>

$\Sigma_1$ works similarly to $\Sigma_0$ with the only difference that performs a 6, 11 and 25 steps right rotation on the input word.   

In [38]:
def S1(x):
    return r_rot(x, 6) ^ r_rot(x, 11) ^ r_rot(x, 25)

### $\sigma_0$ function
<a id="s0"></a>

The $\sigma_0$ function takes one word as an argument. Then it produces a version of it right rotated by 7, one right rotated by 18 and one right shifted by 3. To output the final word performs an exclusive or to all three.

In [39]:
def s0(x):
    return r_rot(x, 7) ^ r_rot(x, 18) ^ (x >> 3)

### $\sigma_1$ function
<a id="s1"></a>

The $\sigma_1$ function works similarly to $\sigma_0$. It produces a version of the input word right rotated by 17 steps, one right rotated by 19 steps and one right shifted by 10 steps. To generate the final word it performs an exclusive or to all three.

In [40]:
def s1(x):
    return r_rot(x, 17) ^ r_rot(x, 19) ^ (x >> 10)

### The main hashing function
<a id="comp"></a>

The hash function takes as an input a message of 64-bytes (512-bits) and performs the following steps:


1. First step will be to initialise the hash values $H$(state registers).


2. The next step is to create the message schedule. To generate the first 16 words of the message schedule we split the input word in chunks of 32 bits. Since the message schedule has to be a list of 64 words we use the $\sigma_0$ and $\sigma_1$ functions to generate the remaining 48 words. Therefore each word $i$, where $i \in (17$ to $64)$ of the message schedule list, will be equal to:
        
    $word_i=((word_{(i-16)}+\sigma_0(word_{(i-2)})+(word_{(i-7)})+\sigma_1(word_{(i-2)}))mod2^{32}$


3. Compression of the message: To compress our message with the hash values we iterate through the message schedule list we have created in the previous step. For each word $i$ ( where $i \in (1$ to $64)$ ) we perform the following steps:

    We create a temporary word $T^{(1)}$ using $\Sigma_1$, $choice()$ functions and the $K$ and $H$ constants, wich equals to:

    $T^{(1)}_i = H_8 + \Sigma_1(H_5) + choice(H_5, H_6, H_7) + K_i + word_i$

    and a temporary word $T^{(2)}$ using the $\Sigma_0$, $majority()$ functions and the $H$ constants, which equals to:

    $T^{(2)}_i = \Sigma_0(H_1) + majority(H_1,H_2, H_3)$
    
    Then we update the $H$ values in the following manner:
    
    $H_8 = H_7, H_7 = H_6, H_6 = H_5, H_5 = (H_4 + T^{(1)}_i)mod2^{32}, H_4 = H_3, H_3 = H_2, H_2 = H_1, H_1 = (T^{(1)}_i + T^{(2)}_i)mod2^{32}$
    
    *(At this stage we are actually dropping the $H_8$ information as it is not assigned)*
    
    This proccess is repeated for every $word_i$ (in the schedule message) and $K_i$. Each round of compression uses the updated $H$ values of the previous iteration.
    
    At the end of this step, we add the updated $H$ values to the initial ones:
    
    $H^{final} = (H^{updated} + H^{initial})mod2^{32}$
    
If there are more multiples of 64-byte message blocks (from our initial input text) this proccess is repeated taking as inital values the $H^{final}$ from the previous compression round. The output of the final round will be the final hash value.

Here is a python implementation of the hash function:

In [41]:
import struct

# Assigning the K constants
K = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
      0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
      0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
      0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
      0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
      0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
      0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
      0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

# Assigning the H constants
H = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]

# Initiating a value to use for mod32 operations
f32 = 0xFFFFFFFF

def msg_compression(message_block, H, K):
    
    # Initiating the Hash values
    a, b, c, d, e, f, g, h = H
    
    # initiating a list with 64 place-holders for the message schedule
    m_schedule =  64 * [None]
    
    # 64 bytes are required, otherwise an error will be raised
    # The first 16 words in the message schedule are populated by the message block chunks
    m_schedule[0:16] = struct.unpack('>16L', message_block)

    # We use the s functions to further populate the message schedule list
    for i in range(16, 64):
        m_schedule[i] = (m_schedule[i-16] + s0(m_schedule[i-15]) + m_schedule[i-7] + s1(m_schedule[i-2])) & f32
    
    
    # Looping through the message schedule and compressing it iteratively
    # to the H constants. During the process we also use the K constants
    # to generate the temporary words T1 and T2.
    for i in range(64):
        T1 = h + S1(e) + choice(e, f, g) + K[i] + m_schedule[i]
        T2 = S0(a) + majority(a, b, c)
        
        # Updating the hash values in the end of each iterration.
        h = g;   g = f;   f = e;   e = (d + T1) & f32;   d = c
        
        c = b;   b = a;   a = (T1 + T2) & f32
    
    # Creating a list with the final hashed values for this message block
    # by adding the initial H constants to the compressed ones
    H = [((x + y) & f32) for x,y in zip(H, [a, b, c, d, e, f, g, h])]
    print (H)

In [42]:
# The function only processes 64 bytes (512-bits) at the time

# A 64 bytes input example
x = b'gtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgtgt'

print ("The input lenght is: ", len(x), "\n\n","And the final H values will be: \n")

msg_compression(x, H, K)

The input lenght is:  64 

 And the final H values will be: 

[4280235720, 1815909234, 2014131107, 4090707777, 2030830240, 2533289756, 2501078905, 2944942141]


## Combining everything to build the SHA-256 algorithm
<a id="comb"></a>

In this section we combine all the previously defined operators and functions to develop the cryptographic algorithm.

A general description of the algorithm and its functionality:

1. The hash function takes an input as a string. To change the text into a binary representation it uses the ascii. Every character in ascii corresponds to a number and then this number is used to produce an 8 bit binary representation of the initial character.

2. The input text is broken down into multiples of 64 bytes (512 bits) to be hashed by the algoritm iteratively. 

3. If the input message is less than 64 bytes or after breaking it into chunks there is a number of characters left that are less than 64 bytes, the algorithm pads these characters to a message block of muliple of 64 bytes and feeds it to the hash function to be consumed. 

4. After the entire input message is compressed, the final hash values are concatenated and converted to hexadecimal to generate the final hash value.

This is a visual representation of how the algorithm works

![Vis](img/sha256.png)

The following sha_256 class is a python implementation of the algorithm:

In [43]:
import copy, struct, binascii


class sha_256(object):
    
    # The K and H constants are initialised
    
    K = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
         0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
         0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
         0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
         0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
         0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
         0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
         0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

    H = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
   

    f32 = 0xFFFFFFFF
    
    compr_msg_lengh = 64
    
    # The construction function for our object is 
    def __init__(self, msg=None):
        
        # We are initiating the instance's lenght to 0
        self.msg_lenght = 0
        
        # We initiate this to represent the instance as byte array
        self.msg_block = b''
        
    
    # We define the methods of our SHA-256 class
    def r_rot(self, x, steps):
        return ((x >> steps) | (x << (32 - steps))) & f32
    
    def S0(self, x):
        return self.r_rot(x, 2) ^ self.r_rot(x, 13) ^ self.r_rot(x, 22)

    def S1(self, x):
        return self.r_rot(x, 6) ^ self.r_rot(x, 11) ^ self.r_rot(x, 25)

    def s0(self, x):
        return self.r_rot(x, 7) ^ self.r_rot(x, 18) ^ (x >> 3)

    def s1(self, x):
        return self.r_rot(x, 17) ^ self.r_rot(x, 19) ^ (x >> 10)
    
    def majority(self, x, y, z):
        return (x & y) ^ (x & z) ^ (y & z)

    def choice(self, x, y, z):
        return (x & y) ^ (~x & z)

    def msg_compression(self, message_block):
        
        self.message_block = message_block
        
        a, b, c, d, e, f, g, h = self.H
        
        m_schedule = 64 * [None]
        
        m_schedule[0:16] = struct.unpack('>16L', self.message_block)

        for i in range(16, 64):
            m_schedule[i] = (m_schedule[i-16] + self.s0(m_schedule[i-15]) + m_schedule[i-7] + self.s1(m_schedule[i-2])) & f32

        for i in range(64):
            T1 = h + self.S1(e) + self.choice(e, f, g) + self.K[i] + m_schedule[i]
            T2 = self.S0(a) + self.majority(a, b, c)

            h = g;   g = f;   f = e;   e = (d + T1) & f32;   d = c
            c = b;   b = a;   a = (T1 + T2) & f32
            
        self.H = [((x + y) & f32) for x, y in zip(self.H, [a, b, c, d, e, f, g, h])]
        
        return self.H

    def update(self, msg):
        
        self.msg = msg
        
        if not self.msg:
            
            return
        
        if type(self.msg) is not bytes:
            
            raise TypeError('The input has to be encoded')
        
        # Length of the message or message residuals every time the update() function runs
        self.msg_lenght = len(self.msg) + self.msg_lenght 
        
        # In the first time the update() runs we turn the message into birary
        # In the second itteration we concatenate the 
        # residuals (cached) with the pad to form a 64 message
        self.msg = self.msg_block + self.msg
        
        # this loop will compress all the 64 multiples of the message.
        # if the lenght//64 has residuals (of lengh less than 64), 
        # the loop will not run and the residuals will be forwaded for padding.
        # Once the padding will convert the residuals to 64 the loop will compress 
        # the message once it is run in the digest() method.
        for i in range(0, len(self.msg) // self.compr_msg_lengh):
            self.msg_compression(self.msg[self.compr_msg_lengh*i : self.compr_msg_lengh*(i + 1)])
        
        # The remainands of 64 are saved to be passed for padding 
        # The update() function will run again inside the digest() method
        # to compress the padded message
        self.msg_block = self.msg[-(len(self.msg) % self.compr_msg_lengh) : ] 
        
    def padding(self, msg_lenght):
        
        length_pad = struct.pack('>Q', msg_lenght << 3)
        mod64 = msg_lenght & 0x3F 

        if mod64 < 56:
            zeros = 55 - mod64
        else:
            zeros = 119 - mod64

        return b'\x80' + (b'\x00' * zeros) + length_pad

    def digest(self):
        # We pad the residuals of 64 for padding
        padded = self.padding(self.msg_lenght)
        
        p = copy.copy(self)
        
        # We pass the pad to the update() function to be merged 
        # with the original or remainder (less than 64)
        # message to be compressed
        p.update(padded)
        
        # The output of the main hash function is concatenated to 
        # produce the final outcome
        return b''.join([struct.pack('!L', i) for i in p.H[:8]])

    def hexdigest(self):  
        # We produce the hexadecimal representation
        # of the output of the digestion
        return binascii.hexlify(self.digest()).decode('ascii')

### Comparison to the python hashlib
<a id="compare"></a>

The following code performs a comparison of the python's hashlib library to our implementation's results.

In [44]:
# Importing python's hashlib
import hashlib 


# Messages to be used for the comparison
messages = ["abc",
            "", 
            "≈355¢∞§¶¶¶", 
            "ª•¶¬˚∆˙", 
            "UoL",
            "Blockchain Programming",
           ("This module is primarily assessed as a chain of courseworks all"
            " leading to the development of a cryptocurrency and its supporting"
            " blockchain.We understand that not everybody comes to the module "
            "with the same backgroundand interests. Therefore, each link of the coursework "
            "chain has two sections: one programming and one theoretical. As a student, "
            "you choose the weightings of the two components: theory/programming ratios "
            "of 30/70, 50/50, or 70/30. When you submit each assignment, you will tell us how "
            "you want your assessments weighted.This first assignment involves exploring"
            " an early cryptographic hash function, called MD2. ‘MD’ stands for Message Digest. "
            "The name refers to the idea that you can use the output of a hash "
            "function as a digest of the input. That is, it is a kind of smaller"
            " description that acts as a proxy for the original thing being hashed. "
            "MD2 was designed by Ron Rivest, the ‘R’ of RSA. MD2 is no longer "
            "usable as a cryptographic hash function: it has been broken. "
            "As technologies improve, it is harder for any function to be secure. "
            "We will not be using MD2 in our development. We will be using the more "
            "secure SHA256. You have already had a feel for how we can work with SHA256 "
            "from one of the activities in this topic. We are asking you to work on MD2 "
            "here because SHA256 is just too complicated to ask you to implement. "
            "The intrepid among you, may wish to give it a go!")]

for i in messages:
    # We hash the message with the hashlib library
    hashlib_hash = hashlib.sha256(i.encode()).hexdigest()
    
    # We hash the same message by using our implementation
    our_hash = sha_256()
    
    # We are using utf-8 as ascii cannot decode some of the characters in the test text list
    our_hash.update(i.encode('utf-8'))
    
    # Validating the results
    print ('Comparison: ',our_hash.hexdigest() == hashlib_hash, '---> The hash value is: ', our_hash.hexdigest())

Comparison:  True ---> The hash value is:  ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Comparison:  True ---> The hash value is:  e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Comparison:  True ---> The hash value is:  6a67b72015b9dcdf22a2578959656d0a09caa7988ac08012542695d4d241bfd7
Comparison:  True ---> The hash value is:  b4d5485dcb0dece3d449b55a8a1b1c49d41215f407bf1f2e1a3875ab7e021fd2
Comparison:  True ---> The hash value is:  add0fcb4f68f2c2ac8f9e869bc5946f4382db632d1460461a81a2435081cf7e9
Comparison:  True ---> The hash value is:  310490fceea756e9c1174f06b6d92e54fa8c351049de74159a8e6feba9fe14fe
Comparison:  True ---> The hash value is:  218c11ab895d76c85c81afa629d5b66b3e6c44f1040fa441fcefed3a081592b2


# References
<a id="ref"></a>

**Bibliography and other sources**

<sup>[1]</sup> Feistel, Horst (1973). "Cryptography and Computer Privacy". Scientific American. 228 (5): 15–23. Bibcode:1973SciAm.228e..15F. doi:10.1038/scientificamerican0573-15

<sup>[2]</sup> Herstein, I. N. (1964), Topics In Algebra, Waltham: Blaisdell Publishing Company, p. 90, ISBN 978-1114541016

<sup>[3]</sup> [The Internet Engineering Task Force (IETF) is an open standards organization](https://www.ietf.org)

<sup>[4]</sup> [Request for Comments, a memorandum on Internet standards](https://tools.ietf.org/rfc/index)

<sup>[5]</sup> Knudsen, L., Mathiassen, J., Muller, F., and Thomsen, S., "Cryptanalysis of MD2", Journal of Cryptology, 23(1):72-90, 2010

<sup>[6]</sup> Frank, P.; Goldstein, S.; Kac, M.; Prager, W.; Szegö, G.; Birkhoff, G., eds. (1964). Selected Papers of Richard von Mises. 2. Providence, Rhode Island: Amer. Math. Soc. pp. 313–334.

<sup>[7]</sup> [RSA: A network security company named after it's co-funders Ron Rivest, Adi Shamir and Leonard Adleman](https://www.rsa.com)

<sup>[8]</sup> [Public-key cryptography standards devised and published by RSA Security](https://en.wikipedia.org/wiki/PKCS)

<sup>[9]</sup> [Salt in cryptography](https://en.wikipedia.org/wiki/Salt_(cryptography)#cite_note-:0-1)

<sup>[10]</sup> Buchanan, W. Cryptography. (Gistrup, Denmark: River Publishers, 2017)

<sup>[11]</sup> "Debian codebase in Google Code". Archived from the original on November 7, 2011. Retrieved 2011-11-08.

<sup>[13]</sup> Hellman, M. (July 1980). "A cryptanalytic time-memory trade-off" (PDF). IEEE Transactions on Information Theory. 26 (4): 401–406. doi:10.1109/tit.1980.1056220. ISSN 0018-9448

<sup>[12]</sup> Schneier, Bruce (January 2000). "A Self-Study Course in Block-Cipher Cryptanalysis". Cryptologia. 24 (1): 18–34. doi:10.1080/0161-110091888754. S2CID 53307028. Archived from the original on 2015-09-11. Retrieved 2011-01-11.

<sup>[14]</sup> Piper, Fred. (2003). Reseacrh in cryptography and security mechanisms. Computers & Security. 22. 22-25. 10.1016/S0167-4048(03)00104-4. 

<sup>[15]</sup> [SHA-256 Description](http://www.iwar.org.uk/comsec/resources/cipher/sha256-384-512.pdf) (This is the same link provided in the coursework but it is not working anymore)

<sup>[16]</sup> [Python bitwise operators](https://wiki.python.org/moin/BitwiseOperators)

<sup>[17]</sup> [Interpret bytes as packed binary data (struct: Python's documentation)](https://docs.python.org/3/library/struct.html#format-characters)

<sup>[18]</sup> [Intel: "Moore's Law and Intel Innovation"](https://www.intel.com/content/www/us/en/history/museum-gordon-moore-law.html)

**Links of sites visited**

[The SHA-256  alogoritm wikipedia](https://en.wikipedia.org/wiki/SHA-2) 

[SHA-2 algorithm and constants Federal Information Processing Standards Publications (FIPS PUBS) ](https://csrc.nist.gov/csrc/media/publications/fips/180/2/archive/2002-08-01/documents/fips180-2.pdf) 

[Multi-Mode Operator for SHA-2 Hash Functions
Ryan Glabb, Laurent Imbert, Graham Jullien, Arnaud Tisserand, Nicolas Veyrat-Charvillon](https://core.ac.uk/download/pdf/191828933.pdf)  

[Binary expaination](https://www.mathsisfun.com/binary-digits.html) 

**Code references**

[SHA-256 implementation in ruby](https://github.com/in3rsha/sha256-animation/blob/master/compression.rb) 

[SHA-256 implementation in ruby (video)](https://www.youtube.com/watch?v=f9EbD6iY9zI)    

[Example of SHA-256 implementation in python](https://github.com/thomdixon/pysha2/blob/master/sha2/sha256.py) 
